# Challenge 1: Properti

## Goals:
- Clean the supplied dataset, designing strategies to deal with lost data in certain variables. 
- Perform a descriptive analysis of the main variables.
- Create new columns based on the given characteristics that may have predictive value.

### Import Libraries
**Numpy** and **Pandas** libraries are used to load and manage the information supplied in the csv file.

**IPython** is used to perform the presentation of the information obtained durign the analysis. 

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

### Load CSV file
Load the dataset specifiying the comma as the delimiter value .

In [2]:
dfprop=pd.read_csv(filepath_or_buffer='properatti.csv',delimiter=',',index_col = 0 )

### Visualize Strucure Information
#### Dataset Structure (Columns - data type)


In [3]:
#ar_columns=np.array(dfprop.columns)
#ar_columns.info()
dfprop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121220 entries, 0 to 121219
Data columns (total 25 columns):
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 non-null float64
price_usd_per_m2              68617 n

### Columns definition

- operation:

- property_type:

- place_name:

- place_with_parent_names:

- country_name:

- state_name:

- geonames_id:

- lat-lon:

- lat:

- lon:

- price:

- currency:

- price_aprox_local_currency:

- price_aprox_usd:

- surface_total_in_m2:

- surface_covered_in_m2:

- price_usd_per_m2:

- price_per_m2:

- floor:

- rooms:

- expenses:

- properati_url:

- description:

- title:

- image_thumbnail:


## Nan values per variable

In [51]:
dfprop[dfprop.isnull().any(axis=1)].isna().sum()

Unnamed: 0                         0
operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

- Analizamos los valores NaN de la columna 'place_name':

In [10]:
dfprop.loc[dfprop['place_name'].isna() == True]

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
6489,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4008968545,-58.63809847",-34.400897,-58.638098,...,240.0,2708.333333,2708.333333,NaN,NaN,NaN,http://www.properati.com.ar/15ui7_venta_casa_t...,"Espectacular casa moderna de una planta, con a...",Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/7/vQjgbHSz1fKXxy...
10201,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.400704,-58.638067",-34.400704,-58.638067,...,300.0,NaN,1783.333333,NaN,5.0,NaN,http://www.properati.com.ar/166p5_venta_casa_t...,Corredor Responsable: SABRINA COCCONI - CSI 62...,VENTA CASA SANTA MARÍA DE TIGRE COUNTRY BARRIO...,https://thumbs4.properati.com/1/FABpKIJMaoAO3t...
11451,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4026880384,-58.6262613379",-34.402688,-58.626261,...,270.0,253.456221,2037.037037,NaN,NaN,NaN,http://www.properati.com.ar/16bpd_venta_casa_t...,MUY LINDA CASA EN EXCELENTE UBICACION!! Superf...,SANTA MARIA DE TIGRE IMPECABLE,https://thumbs4.properati.com/5/fwH9L71hW2rW7V...
14839,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4007994779,-58.6381735719",-34.400799,-58.638174,...,200.0,2000.000000,2400.000000,NaN,NaN,NaN,http://www.properati.com.ar/16tal_venta_casa_b...,Muy linda casa de estilo clásico.-Planta baja;...,Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/1/3ZbPSffTY38y7v...
18622,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4032781195,-58.6263503945",-34.403278,-58.626350,...,340.0,3814.705882,3814.705882,NaN,NaN,NaN,http://www.properati.com.ar/177ee_venta_casa_t...,Casa con increible vista al lago con playa de...,Casa con increible vista al lago con playa y m...,https://thumbs4.properati.com/0/52JCH1PKYCOMe6...
21922,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.399704,-58.638405",-34.399704,-58.638405,...,220.0,NaN,2000.000000,NaN,5.0,NaN,http://www.properati.com.ar/17gy5_venta_casa_s...,Corredor Responsable: Matias Castelli - CUCICB...,Venta Casa - Barrio Santa Maria de Tigre,https://thumbs4.properati.com/8/oO7kmFPYNm8DYr...
23664,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4005161998,-58.6417999185",-34.400516,-58.641800,...,294.0,1632.653061,1632.653061,NaN,NaN,NaN,http://www.properati.com.ar/17mb5_venta_casa_t...,EXCELENTE CASA CLASICA con grandes ventanas y ...,CASA EN VENTA EN SANTA MARIA DE TIGRE CON RENTA,https://thumbs4.properati.com/2/LQCwH6txpFqetn...
24722,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4007292,-58.6377783",-34.400729,-58.637778,...,270.0,1677.215190,1962.962963,NaN,NaN,NaN,http://www.properati.com.ar/17oz1_venta_casa_t...,"EN EXCLUSIVIDAD!!!!!!Lindísima casa, muy lumin...",Venta de casa en Santa María de Tigre,https://thumbs4.properati.com/0/Nhx4NejZaQACap...
38856,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.425087,-58.5796585",-34.425087,-58.579659,...,550.0,NaN,2454.545455,NaN,NaN,NaN,http://www.properati.com.ar/18mcb_venta_casa_t...,SANTA MARIA DE TIGRE- AL RIO - ESPECTACULAR CA...,"Casa (barrio cerrado), Venta",https://thumbs4.properati.com/2/Izq8d5o8ti3kJy...
45970,sell,apartment,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.396074,-58.63738",-34.396074,-58.637380,...,27.0,1727.272727,3518.518519,NaN,1.0,NaN,http://www.properati.com.ar/1958f_venta_depart...,"EDIFICIO ZERENA –

- Vemos que, de acuerdo a 'place_name_with_parents', todos los valores faltantes son 'Tigre'.
- Por lo tanto, procedemos a reemplazarlos:

In [19]:
dfprop['place_name'].fillna( value = 'Tigre', inplace = True)

In [20]:
dfprop[dfprop.isnull().any(axis=1)].isna().sum()

operation                          0
property_type                      0
place_name                         0
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

- Analizamos los valores que tienen 'surface_covered_in_m2', pero no 'surface_total_in_m2'

In [32]:
dfprop_surfaceanalysis = dfprop[(dfprop['surface_total_in_m2'].isna() == True) & (dfprop['surface_covered_in_m2'].isna() == False)]
dfprop_surfaceanalysis[['surface_total_in_m2', 'surface_covered_in_m2']].head()

,surface_total_in_m2,surface_covered_in_m2
68,NaN,72.0
77,NaN,85.0
79,NaN,42.0
87,NaN,170.0
89,NaN,53.0


- Reemplazamos los valores NaN de la columna 'surface_total_in_m2', asumiendo que en estos casos es igual al valor de 'surface_covered_in_m2'

In [39]:
dfprop.fillna({'surface_total_in_m2' : dfprop['surface_covered_in_m2']}, inplace = True)

operation                                                                  sell
property_type                                                         apartment
place_name                                                        Mar del Plata
place_with_parent_names       |Argentina|Buenos Aires Costa Atlántica|Mar de...
country_name                                                          Argentina
state_name                                         Buenos Aires Costa Atlántica
geonames_id                                                         3.43086e+06
lat-lon                                                 -38.0034731,-57.5501571
lat                                                                    -38.0035
lon                                                                    -57.5502
price                                                                    150000
currency                                                                    USD
price_aprox_local_currency              

Volvemos a constatar la cantidad de valores nulos en el dataframe:

In [40]:
dfprop[dfprop.isnull().any(axis=1)].isna().sum()

operation                          0
property_type                      0
place_name                         0
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            12369
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

Repetimos la operación para los casos en los que no tenemos valor de 'surface_covered_in_m2', completando con el valor de 'surface_total_in_m2':

In [42]:
dfprop_surfaceanalysis = dfprop[(dfprop['surface_covered_in_m2'].isna() == True) & (dfprop['surface_total_in_m2'].isna() == False)]
dfprop_surfaceanalysis[['surface_total_in_m2', 'surface_covered_in_m2']].head()

,surface_total_in_m2,surface_covered_in_m2
5,53.0,NaN
10,1514.0,NaN
15,164.0,NaN
66,38.0,NaN
67,220.0,NaN


In [43]:
dfprop.fillna({'surface_covered_in_m2' : dfprop['surface_total_in_m2']}, inplace = True)

operation                                                                  sell
property_type                                                             house
place_name                                                         Gualeguaychú
place_with_parent_names                     |Argentina|Entre Ríos|Gualeguaychú|
country_name                                                          Argentina
state_name                                                           Entre Ríos
geonames_id                                                         3.43366e+06
lat-lon                                                  -33.0140714,-58.519828
lat                                                                    -33.0141
lon                                                                    -58.5198
price                                                                       NaN
currency                                                                    NaN
price_aprox_local_currency              

In [44]:
dfprop[dfprop.isnull().any(axis=1)].isna().sum()

operation                          0
property_type                      0
place_name                         0
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            12369
surface_covered_in_m2          12369
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64